# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
list(school_data)

['School ID', 'school_name', 'type', 'size', 'budget']

## District Summary

In [3]:
no_schools = school_data_complete['School ID'].nunique()
no_students = school_data_complete['Student ID'].nunique()
total_budget = school_data_complete['budget'].sum()
avg_math = round(school_data_complete['math_score'].mean(),2)
avg_read = round(school_data_complete['reading_score'].mean(),2)
math_passing = school_data_complete.loc[school_data_complete["math_score"] >= 70]
math_passing_count = len(math_passing["math_score"])
math_pass_pct = round(((math_passing_count)*100)/no_students,2)
read_passing = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
read_passing_count = len(read_passing["reading_score"])
read_pass_pct = round((read_passing_count*100)/no_students,2)
overall = round((avg_math + avg_read)/2, 2)

In [4]:
district_summary = pd.DataFrame(
    {"# of Schools": [str(no_schools)],
     "# of Students": [str(no_students)],
     "Total Budget": ["$" + str(total_budget)],
     "avg_math": [str(avg_math)],
     "avg_read": [str(avg_read)],
     "% Passing Math":[str(math_pass_pct) + " %"],
     "% Passing Reading":[str(read_pass_pct) + " %"],
     "Overall": [str(overall) + " %"]
    })
district_summary

,# of Schools,# of Students,Total Budget,avg_math,avg_read,% Passing Math,% Passing Reading,Overall
0,15,39170,$82932329558,78.99,81.88,74.98 %,85.81 %,80.44 %


## School Summary

In [5]:
schools = school_data_complete.set_index('school_name').groupby(['school_name'])
sch_types = school_data.set_index('school_name')['type']
stu_per_sch = schools['Student ID'].count()
sch_budget = school_data.set_index('school_name')['budget']
stu_budget = round(school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size'],2)
avg_math = round(schools['math_score'].mean(),2)
avg_read = round(schools['reading_score'].mean(),2)
pass_math = round((school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch*100),2) 
pass_read = round((school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch*100),2)
overall = round((school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch*100),2)



In [6]:
sch_summary = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Per Student Budget": stu_budget,
    "Total School Budget": sch_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    "# Passing Math": pass_math,
    "% Passing Read": pass_read,
    "% Overall Passing Rate": overall
})
sch_summary.head()

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,# Passing Math,% Passing Read,% Overall Passing Rate
Bailey High School,District,4976,628.0,3124928,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,582.0,1081356,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,639.0,1884411,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,644.0,1763916,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,625.0,917500,83.35,83.82,93.39,97.14,90.60


In [7]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [8]:
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## Top Performing Schools (By Passing Rate)

In [9]:
top_schools = sch_summary.sort_values(by='% Overall Passing Rate', ascending=False)
top_schools.head()

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,# Passing Math,% Passing Read,% Overall Passing Rate
Cabrera High School,Charter,1858,582.0,1081356,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,638.0,1043130,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,625.0,917500,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,578.0,1319574,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,609.0,585858,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By Passing Rate)

In [10]:
top_schools = sch_summary.sort_values(by='% Overall Passing Rate', ascending=True)
top_schools.head()

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,# Passing Math,% Passing Read,% Overall Passing Rate
Rodriguez High School,District,3999,637.0,2547363,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,639.0,1884411,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,655.0,1910635,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,652.0,3022020,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,650.0,3094650,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

In [11]:
math_9 = round(school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby("school_name")["math_score"].mean(),2)
math_10 = round(school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby("school_name")["math_score"].mean(),2)
math_11 = round(school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby("school_name")["math_score"].mean(),2)
math_12 = round(school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby("school_name")["math_score"].mean(),2)

In [12]:
math_score_grade = pd.DataFrame({
    "9th": math_9,
    "10th": math_10,
    "11th": math_11,
    "12th": math_12,
})

math_score_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36


## Reading Score by Grade 

In [13]:
read_9 = round(school_data_complete.loc[school_data_complete["grade"] == "9th"].groupby("school_name")["reading_score"].mean(),2)
read_10 = round(school_data_complete.loc[school_data_complete["grade"] == "10th"].groupby("school_name")["reading_score"].mean(),2)
read_11 = round(school_data_complete.loc[school_data_complete["grade"] == "11th"].groupby("school_name")["reading_score"].mean(),2)
read_12 = round(school_data_complete.loc[school_data_complete["grade"] == "12th"].groupby("school_name")["reading_score"].mean(),2)

In [14]:
read_score_grade = pd.DataFrame({
    "9th": read_9,
    "10th": read_10,
    "11th": read_11,
    "12th": read_12,
})

read_score_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01


## Scores by School Spending

In [22]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

#sch_spending_scores = sch_summary[["Per Student Budget", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

sch_spending_scores["Spending Bins"] = pd.cut(sch_spending_scores["Per Student Budget"], spending_bins, labels=group_names)
sch_spending_scores.head()


NameError: name 'sch_spending_scores' is not defined

## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
school_size = school_data_complete.set_index('school_name').groupby(['size'])
stu_per_sch = school_size['Student ID'].count()
avg_math_size = round(school_size['math_score'].mean(),2)
avg_read_size = round(school_size['reading_score'].mean(),2)
pass_math_size = round((school_data_complete[school_data_complete['math_score'] >= 70].groupby('size')['Student ID'].count()/stu_per_sch*100),2) 
pass_read_size = round((school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size')['Student ID'].count()/stu_per_sch*100),2)
overall_size = round((school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size')['Student ID'].count()/stu_per_sch*100),2)


In [18]:
sch_size_summary = pd.DataFrame({
    "Average Math Score": avg_math_size,
    "Average Reading Score": avg_read_size,
    "# Passing Math": pass_math_size,
    "% Passing Read": pass_read_size,
    "% Overall Passing Rate": overall_size
})
sch_size_summary.head()



,Average Math Score,Average Reading Score,# Passing Math,% Passing Read,% Overall Passing Rate
size,,,,,
427,83.80,83.81,92.51,96.25,89.23
962,83.84,84.04,94.59,95.95,90.54
1468,83.35,83.82,93.39,97.14,90.60
1635,83.42,83.85,93.27,97.31,90.95
1761,83.36,83.73,93.87,95.85,89.89


In [24]:
list(sch_size_summary)

['Average Math Score',
 'Average Reading Score',
 '# Passing Math',
 '% Passing Read',
 '% Overall Passing Rate']

In [25]:
bin = [0,1000,2000,3000,4000]
group_labels = ["Less than 1000", "1000 - 2000", "2000 - 3000", "More than 4000"]

#sch_size_summary["Size Groups"] = pd.cut(sch_size_summary["size"], bin, labels = group_labels)

## Scores by School Type

* Perform the same operations as above, based on school type.

In [20]:
school_type = school_data_complete.set_index('school_name').groupby(['type'])
stu_per_sch_type = school_type['School ID'].count()
avg_math_type = round(school_type['math_score'].mean(),2)
avg_read_type = round(school_type['reading_score'].mean(),2)
pass_math_type = round((school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/stu_per_sch*100),2) 
pass_read_type = round((school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/stu_per_sch*100),2)
overall_type = round((school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/stu_per_sch*100),2)


In [21]:
sch_type_summary = pd.DataFrame({
    "Average Math Score": avg_math_type,
    "Average Reading Score": avg_read_type,
    "# Passing Math": pass_math_type,
    "% Passing Read": pass_read_type,
    "% Overall Passing Rate": overall_type
})
sch_type_summary.head(2)

,Average Math Score,Average Reading Score,# Passing Math,% Passing Read,% Overall Passing Rate
Charter,83.41,83.90,NaN,NaN,NaN
District,76.99,80.96,NaN,NaN,NaN
